## Pulling Census Data from Census API for CMS DASH TC #1

Kevin Chamberlain 
5/27/2021

**Purpose of this notebook:** Pull Census ACS social, economic, housing, and demographic data from 5-yr tables: DP02, DP03, DP04, and DP05 for CMS DASH Tech Challenge #1

**Setup:** Import packages and initiate API connection

In [34]:
## Check system version
import sys
print (sys.version)
## Python 3.7.9

3.7.9 (default, Aug 31 2020, 17:10:11) [MSC v.1916 64 bit (AMD64)]


In [35]:
## Import required packages
from census import Census # Package to utilize Census API
import pandas as pd # Python's core datafrmae package
from pandas.io.json import json_normalize # Used to convert API pulls 
                                          # to dataframes

Datasets available through the census package:
* ACS 5-Year Estimates dating back to 2010
* ACS 1-Year Estimates and data profiles dating back to 2012
* Census Summary Files for 1990, 2000, and 2010

Set ACS API Key and Year. Using 2019, 5-yr data. 

In [36]:
c = Census("1da477885b9ba806514e53f06b5168e519c5c987", year = 2017) 
## Establishes API connection to the Census

Confirm available tables. Will need to transform the data pull from JSON format into a dataframe.

In [37]:
## Create JSON of available tables for 2019 5-year ACS Estimates
acs_tables = c.acs5dp.tables()
## without doing any transformation - JSON format key-value pairs

In [38]:
## Transform JSON format into a dataframe usin the json_normalize package
df_acs_tables = pd.json_normalize(acs_tables)
df_acs_tables.head(5)

,name,description,variables
0,DP04,SELECTED HOUSING CHARACTERISTICS,https://api.census.gov/data/2017/acs/acs5/prof...
1,DP05,ACS DEMOGRAPHIC AND HOUSING ESTIMATES,https://api.census.gov/data/2017/acs/acs5/prof...
2,DP02PR,SELECTED SOCIAL CHARACTERISTICS IN PUERTO RICO,https://api.census.gov/data/2017/acs/acs5/prof...
3,DP02,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,https://api.census.gov/data/2017/acs/acs5/prof...
4,DP03,SELECTED ECONOMIC CHARACTERISTICS,https://api.census.gov/data/2017/acs/acs5/prof...


Review the following link to better understand the ACS table structures: [ACS Table Groupings](https://api.census.gov/data/2017/acs/acs5.html 'ACS Table Groupings')  

You can only access Detailed Tables through this API connection (those tables that start with B or C). You cannot access Subject Tables. That isn't a major issue, however, as Subject Tables are essentially aggregations of Detailed Tables.

**Geographic Levels Available for 5-year ACS:**  
* State
* State - County
* State - County - Blockgroup
* State - County - Subdivision
* **State - County - Tract**
* State - Place
* State - Congressional District
* State - Upper Legislative District
* State - Lower Level District
* Zipcode
* US (National)

I have highlighted State-County-Tract because that's what I'll be demonstrating. One of the challenges with using this level of data is that the package (and maybe the Census API) restricts tract-level data calls to one state at a time. Therefore, if you're trying to pull tract-level data for all states, you either need to iterate through 50 data calls...or write a function!

**Aside - What is a census tract?** An area roughly equivalent to a neighborhood established by the Bureau of Census for analyzing populations. They generally encompass a population between 2,500 to 8,000 people, but certain census tracts do have populations of 0 (bodies of water, airports, mountain ranges, etc.) Census tracts do not adhere to political boundaries.

**Create function for API data calls**

Note the API can be a bit finnicky at times and not work the first time around (doesn't happen often). If it does happen, re-run the code cell and it should work the second time.

In [67]:
def acs_api_pull(variables, states):
    '''Iterate through ACS API data pulls by state and variables identified. 
    Then combine those separate data files into one list. Return a dataframe'''
    output_list = []
    for s in states:
        output_list += c.acs5.state_county_tract(variables,
                                                s, Census.ALL, Census.ALL)
    output_df = json_normalize(output_list)
    return output_df

acs5.state_county_tract comes from the census package. It is the command used to pull 5-year ACS data at the tract level. The c at the beginning is our API connection initalized above. The inputs to the command (not the function) include:
* Variables we want pulled (variables)
* States to be included (s)
* Counties to be included (in this case all - Census.ALL)
* Tracts to be included (in this case all - Census.ALL)

In order for the above function to pull all state data at once, you will need to import a data file witl all state FIPS codes. DC and Puerto Rico are included in this list

In [68]:
## Need to import a data file with all state FIPS codes to use with data call
state_fips = pd.read_csv('C:\\Users\\30640\\OneDrive - ICF\\Reference_Data\\us_states_v3.csv', dtype = {'state_fips': str})
state_fips.state_fips = state_fips.state_fips.str.zfill(2)
state_fips.head(5)
## If you want to exclude certain states from your data pull,
## you can filter them out here.

,state_fips,state_abbrev,state_upper,state_name
0,01,AL,ALABAMA,Alabama
1,02,AK,ALASKA,Alaska
2,04,AZ,ARIZONA,Arizona
3,05,AR,ARKANSAS,Arkansas
4,06,CA,CALIFORNIA,California


In [69]:
## Need to convert FIPS codes into a list for use with the function
state_fips_list = list(state_fips.state_fips)

In [94]:
df_built_raw = pd.read_csv('C:\\Users\\30640\\Desktop\\ACSDP5Y2019.DP05_2021-05-28T121533\\ACSDP5Y2019.DP05_data_with_overlays_2021-05-27T095905.csv', low_memory=False)

In [95]:
df_built_raw = df_built_raw.filter(['GEO_ID','NAME','DP05_0001E','DP05_0033E','DP05_0037E','DP05_0038E','DP05_0039E',
                            'DP05_0044E','DP05_0045E','DP05_0059E','DP05_0071E','DP05_0078E',
                            'DP05_0002E','DP05_0003E'])

**Census API Data Pull: Select Demographic Characteristics from Table DP05**  

In [91]:
## Pulling the estimate values for listed variables from Census API. 
df_built_raw = acs_api_pull(['DP05_0001E','DP05_0033E','DP05_0037E','DP05_0038E','DP05_0039E',
                            'DP05_0044E','DP05_0045E','DP05_0059E','DP05_0071E','DP05_0078E',
                            'DP05_0002E','DP05_0003E'], state_fips_list)

C:\Users\30640\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [96]:
## Get number of rows (counties) and columns (variables)
df_built_raw.shape

(3220, 14)

In [105]:
# Get last 5 from GEO_ID and create new county FIPS column
df_built_raw['FIPS'] = df_built_raw['GEO_ID'].str[-5:]
df_built_raw.head(1)

,GEO_ID,NAME,DP05_0001E,DP05_0033E,DP05_0037E,DP05_0038E,DP05_0039E,DP05_0044E,DP05_0045E,DP05_0059E,DP05_0071E,DP05_0078E,DP05_0002E,DP05_0003E,FIPS
0,0500000US01001,"Autauga County, Alabama",55380,55380,42527,10538,140,573,0,464,1565,10524,26934,28446,01001


In [106]:
## Make copy of data frame to avoid making the API call again
df_built_clean = df_built_raw.copy()

In [108]:
## Review data frame to confirm no null values
df_built_clean.isnull().sum()

GEO_ID        0
NAME          0
DP05_0001E    0
DP05_0033E    0
DP05_0037E    0
DP05_0038E    0
DP05_0039E    0
DP05_0044E    0
DP05_0045E    0
DP05_0059E    0
DP05_0071E    0
DP05_0078E    0
DP05_0002E    0
DP05_0003E    0
FIPS          0
dtype: int64

In [49]:
## Create geo_id
df_built_clean['geo_id'] = df_built_clean.state + df_built_clean.county + df_built_clean.tract

In [50]:
## Rename columns
df_built_clean.rename(columns = {'B25034_001E' : 'est_total',
                                 'B25034_011E' : 'est_1940'},
                                inplace = True)

In [51]:
## Reorder columns
df_built_clean = df_built_clean[['geo_id', 'state', 'county', 'tract', 'est_total',
                                  'est_1940']]

In [52]:
## Check for nulls
df_built_clean.est_total.isnull().value_counts() # No nulls

False    72877
Name: est_total, dtype: int64

In [53]:
## Count number of tracts with 0 homes
(df_built_clean.est_total==0).sum() # 852 tracts with 0 homes

792

In [54]:
## Create percent variable
df_built_clean['pct_1940'] = (df_built_clean.est_1940/df_built_clean.est_total) * 100

In [55]:
## Change missing percentages to 0's
df_built_clean['pct_1940'].fillna(0, inplace = True)
df_built_clean.pct_1940.isnull().value_counts() # 0 instances of missing pct

False    72877
Name: pct_1940, dtype: int64

In [56]:
# Review
df_built_clean.head(10)

,geo_id,state,county,tract,est_total,est_1940,pct_1940
0,01073000300,01,073,000300,1161.0,183.0,15.762274
1,01073000400,01,073,000400,1631.0,516.0,31.637032
2,01073003200,01,073,003200,642.0,97.0,15.109034
3,01073003500,01,073,003500,1240.0,321.0,25.887097
4,01073003700,01,073,003700,2378.0,510.0,21.446594
5,01073004701,01,073,004701,2719.0,1176.0,43.251195
6,01073005600,01,073,005600,2523.0,19.0,0.753072
7,01073010301,01,073,010301,1363.0,452.0,33.162142
8,01073010602,01,073,010602,3007.0,792.0,26.338543
9,01073011500,01,073,011500,2352.0,30.0,1.275510


In [57]:
## Last check for any missing values
df_built_clean.isnull().sum() # No missing values across the board
# Looks good

geo_id       0
state        0
county       0
tract        0
est_total    0
est_1940     0
pct_1940     0
dtype: int64

**API Data Pull: Family Poverty**  
Review link for additional information and data dictionary: [ACS Family Poverty](https://api.census.gov/data/2017/acs/acs5/groups/B17010.html 'ACS Family Poverty')

In [58]:
## Pulling the estimate values for total families
## and families below poverty with/without children. 
df_poverty_raw = acs_api_pull(['B17010_001E', 'B17010_002E', 'B17010_004E',
                              'B17010_005E', 'B17010_011E', 'B17010_012E',
                              'B17010_017E', 'B17010_018E'], state_fips_list)
## Only inputs needed are the variables of interest and
## the list of state FIPS codes

C:\Users\30640\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [59]:
df_poverty_raw.head()

,B17010_001E,B17010_002E,B17010_004E,B17010_005E,B17010_011E,B17010_012E,B17010_017E,B17010_018E,state,county,tract
0,436.0,162.0,35.0,0.0,46.0,0.0,49.0,23.0,01,073,000300
1,840.0,245.0,0.0,0.0,0.0,0.0,169.0,34.0,01,073,000400
2,341.0,143.0,17.0,0.0,0.0,0.0,91.0,11.0,01,073,003200
3,540.0,146.0,0.0,0.0,0.0,0.0,123.0,35.0,01,073,003500
4,1084.0,273.0,58.0,0.0,23.0,0.0,108.0,44.0,01,073,003700


Below is additional code I wrote to clean and add metrics to the data frame. Feel free to review and test.

In [60]:
## Make copy of dataframe to avoid having to perform API call again
df_poverty_clean= df_poverty_raw.copy()

In [61]:
## Create geo_id
df_poverty_clean['geo_id'] = df_poverty_clean.state + df_poverty_clean.county + df_poverty_clean.tract

In [26]:
## Rename columns
df_poverty_clean.rename(columns = {'B17010_001E': 'fam_num',
                                   'B17010_002E': 'fam_pov_num',
                                   'B17010_004E': 'est_married_below_children_all',
                                   'B17010_005E': 'est_married_below_children_under5',
                                   'B17010_011E': 'est_male_below_children_all',
                                   'B17010_012E': 'est_male_below_children_under5',
                                   'B17010_017E': 'est_female_below_children_all',
                                   'B17010_018E': 'est_female_below_children_under5'},
                       inplace = True)

In [27]:
## Reorder columns
df_poverty_clean = df_poverty_clean[['geo_id', 'state', 'county', 'tract',
                                    'fam_num', 'fam_pov_num', 'est_married_below_children_all',
                                    'est_married_below_children_under5',
                                    'est_male_below_children_all',
                                    'est_male_below_children_under5',
                                    'est_female_below_children_all',
                                    'est_female_below_children_under5']]

We are primarily interested in fam_num and fam_pov_num to calculate the percentage of families living within the census tract that had a poverty status within the last 12 months, regardless of the presence or age of children.

In [28]:
## Review data frame
df_poverty_clean.isnull().sum()
## No null values

geo_id                               0
state                                0
county                               0
tract                                0
fam_num                              0
fam_pov_num                          0
est_married_below_children_all       0
est_married_below_children_under5    0
est_male_below_children_all          0
est_male_below_children_under5       0
est_female_below_children_all        0
est_female_below_children_under5     0
dtype: int64

In [29]:
## Check for total family counts of 0
(df_poverty_clean.fam_num==0).sum()
## 928 census tracts with family counts of 0

928

In [30]:
## Check for total family counts <= 10
(df_poverty_clean.fam_num <= 10).sum()
## 1,037 census tracts with family counts <= 10

1037

In [31]:
## Create family in poverty percent column, regardless of whether or not there are children in family
df_poverty_clean['pov_all'] = (df_poverty_clean.fam_pov_num/df_poverty_clean.fam_num) * 100

In [32]:
## Change nulls to 0's
df_poverty_clean.pov_all.fillna(0, inplace = True) # Replace null with 0%
df_poverty_clean.pov_all.isnull().value_counts() # Now 0 with missing values

False    74001
Name: pov_all, dtype: int64

In [33]:
## Create family with children in poverty percent column
df_poverty_clean['pov_child'] = ((df_poverty_clean.est_married_below_children_all +
                                            df_poverty_clean.est_male_below_children_all +
                                            df_poverty_clean.est_female_below_children_all) /
                                            df_poverty_clean.fam_num) * 100

In [34]:
## Change nulls to 0's
df_poverty_clean.pov_child.fillna(0, inplace = True) # Replace null with 0%
df_poverty_clean.pov_child.isnull().value_counts() # Now 0 with missing values

False    74001
Name: pov_child, dtype: int64

In [35]:
## Create families with only children under 5 below poverty percent column
df_poverty_clean['pov_child5'] = ((df_poverty_clean.est_married_below_children_under5 +
                                            df_poverty_clean.est_male_below_children_under5 +
                                            df_poverty_clean.est_female_below_children_under5) /
                                            df_poverty_clean.fam_num) * 100

In [36]:
## Change nulls to 0's
df_poverty_clean.pov_child5.fillna(0, inplace = True) # Replace null with 0%
df_poverty_clean.pov_child5.isnull().value_counts() # Now 0 with missing values

False    74001
Name: pov_child5, dtype: int64

In [37]:
# Check dataframe once more for missing values
df_poverty_clean.isnull().sum() # No missing values

geo_id                               0
state                                0
county                               0
tract                                0
fam_num                              0
fam_pov_num                          0
est_married_below_children_all       0
est_married_below_children_under5    0
est_male_below_children_all          0
est_male_below_children_under5       0
est_female_below_children_all        0
est_female_below_children_under5     0
pov_all                              0
pov_child                            0
pov_child5                           0
dtype: int64

**Add state, county, and tract names to datafile**

In [15]:
acs_names = acs_api_pull('NAME', state_fips_list)

In [39]:
acs_names.head()

,NAME,state,county,tract
0,"Census Tract 100.01, Bibb County, Alabama",01,007,010001
1,"Census Tract 100.02, Bibb County, Alabama",01,007,010002
2,"Census Tract 100.03, Bibb County, Alabama",01,007,010003
3,"Census Tract 100.04, Bibb County, Alabama",01,007,010004
4,"Census Tract 2319, Macon County, Alabama",01,087,231900


In [40]:
## Create multiple columns for state, county, and tract names
acs_names[['n_tract', 'n_county', 'n_state']] = acs_names['NAME'].str.split(',', expand = True)
acs_names.head()

,NAME,state,county,tract,n_tract,n_county,n_state
0,"Census Tract 100.01, Bibb County, Alabama",01,007,010001,Census Tract 100.01,Bibb County,Alabama
1,"Census Tract 100.02, Bibb County, Alabama",01,007,010002,Census Tract 100.02,Bibb County,Alabama
2,"Census Tract 100.03, Bibb County, Alabama",01,007,010003,Census Tract 100.03,Bibb County,Alabama
3,"Census Tract 100.04, Bibb County, Alabama",01,007,010004,Census Tract 100.04,Bibb County,Alabama
4,"Census Tract 2319, Macon County, Alabama",01,087,231900,Census Tract 2319,Macon County,Alabama


In [110]:
df_built_clean

,GEO_ID,NAME,DP05_0001E,DP05_0033E,DP05_0037E,DP05_0038E,DP05_0039E,DP05_0044E,DP05_0045E,DP05_0059E,DP05_0071E,DP05_0078E,DP05_0002E,DP05_0003E,FIPS
0,0500000US01001,"Autauga County, Alabama",55380,55380,42527,10538,140,573,0,464,1565,10524,26934,28446,01001
1,0500000US01003,"Baldwin County, Alabama",212830,212830,183471,19718,1645,1969,505,552,9711,19525,103496,109334,01003
2,0500000US01005,"Barbour County, Alabama",25361,25361,11869,12066,82,134,65,25,1105,12031,13421,11940,01005
3,0500000US01007,"Bibb County, Alabama",22493,22493,17272,5014,30,27,27,80,579,4981,12150,10343,01007
4,0500000US01009,"Blount County, Alabama",57681,57681,55062,928,46,212,43,179,5342,881,28495,29186,01009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,0500000US72145,"Vega Baja Municipio, Puerto Rico",52192,52192,43427,2589,64,9,9,302,50491,9,24963,27229,72145
3216,0500000US72147,"Vieques Municipio, Puerto Rico",8642,8642,4730,651,8,14,14,88,8204,0,4277,4365,72147
3217,0500000US72149,"Villalba Municipio, Puerto Rico",22403,22403,12713,773,0,0,0,6172,22346,18,10856,11547,72149
3218,0500000US72151,"Yabucoa Municipio, Puerto Rico",33499,33499,6244,26418,0,11,0,225,33472,0,16202,17297,72151


In [113]:
cols = df_built_clean.columns.tolist()
cols

['GEO_ID',
 'NAME',
 'DP05_0001E',
 'DP05_0033E',
 'DP05_0037E',
 'DP05_0038E',
 'DP05_0039E',
 'DP05_0044E',
 'DP05_0045E',
 'DP05_0059E',
 'DP05_0071E',
 'DP05_0078E',
 'DP05_0002E',
 'DP05_0003E',
 'FIPS']

In [114]:
cols = cols[-1:] + cols[:-1]

In [115]:
df_built_clean = df_built_clean[cols]

In [116]:
df_DP05_built_clean

,FIPS,GEO_ID,NAME,DP05_0001E,DP05_0033E,DP05_0037E,DP05_0038E,DP05_0039E,DP05_0044E,DP05_0045E,DP05_0059E,DP05_0071E,DP05_0078E,DP05_0002E,DP05_0003E
0,01001,0500000US01001,"Autauga County, Alabama",55380,55380,42527,10538,140,573,0,464,1565,10524,26934,28446
1,01003,0500000US01003,"Baldwin County, Alabama",212830,212830,183471,19718,1645,1969,505,552,9711,19525,103496,109334
2,01005,0500000US01005,"Barbour County, Alabama",25361,25361,11869,12066,82,134,65,25,1105,12031,13421,11940
3,01007,0500000US01007,"Bibb County, Alabama",22493,22493,17272,5014,30,27,27,80,579,4981,12150,10343
4,01009,0500000US01009,"Blount County, Alabama",57681,57681,55062,928,46,212,43,179,5342,881,28495,29186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,72145,0500000US72145,"Vega Baja Municipio, Puerto Rico",52192,52192,43427,2589,64,9,9,302,50491,9,24963,27229
3216,72147,0500000US72147,"Vieques Municipio, Puerto Rico",8642,8642,4730,651,8,14,14,88,8204,0,4277,4365
3217,72149,0500000US72149,"Villalba Municipio, Puerto Rico",22403,22403,12713,773,0,0,0,6172,22346,18,10856,11547
3218,72151,0500000US72151,"Yabucoa Municipio, Puerto Rico",33499,33499,6244,26418,0,11,0,225,33472,0,16202,17297
